In [ ]:
import geopandas as gpd
from AutoCordon.buffer_zone import BufferZone
from AutoCordon.buffer_zone_graph import get_subgraphs, make_buffer_zone_graph
from AutoCordon.prune_edges import get_closure_edges
from pygeos.io import from_shapely
import pickle as pkl
import momepy as mm
import numpy as np
import networkx as nx

In [ ]:
roads_path = r"..\tests\data\SU_RoadLink.shp"
distance = 550
distance_max = 750
distance_wider = distance_max * 2
centre = (442000, 112000)

In [ ]:
roads = gpd.read_file(roads_path)
roads = roads.explode().reset_index(drop=True)
roads_simplified = mm.remove_false_nodes(roads)
road_lines = from_shapely(roads_simplified.geometry)

In [ ]:
cordon_bz = BufferZone(centre, distance, distance_max)

remaining_roads = cordon_bz.get_intersecting_lines(road_lines)
remaining_roads_gdf = gpd.GeoDataFrame(geometry=road_lines)
interior_closure_points = cordon_bz.get_intersecting_perimeter_points("interior",
                                                                road_lines)
exterior_closure_points = cordon_bz.get_intersecting_perimeter_points("exterior",
                                                                road_lines)

In [ ]:
wider_bz = BufferZone(centre, distance_max, distance_wider)

remaining_roads_wider = wider_bz.get_intersecting_lines(road_lines)
remaining_roads_gdf_wider = gpd.GeoDataFrame(geometry=road_lines)
interior_closure_points_wider = wider_bz.get_intersecting_perimeter_points("interior",
                                                                road_lines)
exterior_closure_points_wider = wider_bz.get_intersecting_perimeter_points("exterior",
                                                                road_lines)

In [ ]:
all_remaining_roads = np.concatenate((remaining_roads, remaining_roads_wider))
all_remaining_roads

In [ ]:
graph = make_buffer_zone_graph(centre, all_remaining_roads)

In [ ]:
from AutoCordon.prune_edges import get_closure_edges

graph = nx.Graph(nx.to_undirected(graph))
closure_edges = get_closure_edges(graph)

In [ ]:
closure_edges

In [ ]:
subgraphs = get_subgraphs(graph, interior_closure_points, exterior_closure_points,
                          exterior_closure_points_wider)

In [ ]:
subgraphs

In [ ]:
from cut_edges import get_all_closures, plot_closures

data = get_all_closures(subgraphs)

In [ ]:
starting_edges_gdf = gpd.GeoDataFrame(data["starting_edges"])
sources_gdf = gpd.GeoDataFrame(data["sources"])
sinks_gdf = gpd.GeoDataFrame(data["sinks"])
closed_edges_calc_gdf = gpd.GeoDataFrame(data["closed_edges_calc"])
closed_edges_hybrid_gdf = gpd.GeoDataFrame(data["closed_edges_hybrid"])

plot_closures(roads,
              starting_edges_gdf,
              sources_gdf,
              sinks_gdf,
              closed_edges_calc_gdf,
              closed_edges_hybrid_gdf,
              None, save=False)

for subgraph_id in subgraphs:
        plot_closures(roads,
                      starting_edges_gdf[starting_edges_gdf["subgraph_id"] == subgraph_id],
                      sources_gdf[sources_gdf["subgraph_id"] == subgraph_id],
                      sinks_gdf[sinks_gdf["subgraph_id"] == subgraph_id],
                      closed_edges_calc_gdf[closed_edges_calc_gdf["subgraph_id"] == subgraph_id],
                      closed_edges_hybrid_gdf[closed_edges_hybrid_gdf["subgraph_id"] == subgraph_id],
                      subgraph_id, save=False)